In [ ]:
# Visualisation libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['text.usetex'] = True
plt.rcParams["font.family"] = "Times"

# Auxiliary libraries for the computation and plot of the results
import sys
sys.path.insert(0, "./Background_Scripts")
from Background_Scripts.ttensor import *
from Background_Scripts.utils import *
from Background_Scripts.plot_functions import *
from Background_Scripts.HCP_Data_Vis_Schaefer_100Parcels import *

# Magic command to load watermark
%load_ext watermark


In [3]:
# Possibility to stop warnings
import warnings

warnings.filterwarnings('ignore') 

In [ ]:
# Print versions
%watermark --author "Breno & Fernando" --date --time --python --machine --iversion --watermark --packages meshio,scikit-learn,jupyterlab,notebook

In [5]:
# Definition of file directories
correlation_matrix_dir = './Schaefer_100Parcels_Atlas/Averaged_Correlation_Matrix.txt'
triplet_weights_dir = './Schaefer_100Parcels_Atlas/Averaged_Zscored_Triplet_Weights.csv'

Load the hyperedges weighths in the file and generate *ttensor* objects related to the simmetrized adjacency tensors $\mathcal{M}_s$ and $\mathcal{I}_s$

In [7]:
As_ii, As_tc = get_symmetrized_ttensors(triplet_weights_dir)

Defining $\mathbf{A}$, $\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(0)}$ and $\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(0)}$, which are the adjacency matrices of $\mathcal{G}$, $\mathcal{G}_{II}^{(0)}$ and $\mathcal{G}_{TC}^{(0)}$, respectively.

In [8]:
A = np.genfromtxt(correlation_matrix_dir)
Aii_0 = np.abs(As_ii.fft[:,:,0])
Atc_0 = np.abs(As_tc.fft[:,:,0])

Running the spectral clustering algorithm on $\mathbf{A}$, $\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(0)}$ and $\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(0)}$ for $K\in\{2,3,\cdots,30\}$.

In [9]:
k_clusters = range(2,31)

sc_models_G, silhouette_score_sc_models_G, sc_embeddings_G = graph_spectral_clustering(A, k_clusters, shift_operator='laplacian', norm_type='sym', eig_type='U')
sc_models_Gii_0, silhouette_score_sc_models_Gii_0, sc_embeddings_Gii_0 = graph_spectral_clustering(Aii_0, k_clusters, shift_operator='laplacian', norm_type='sym', eig_type='U')
sc_models_Gtc_0, silhouette_score_sc_models_Gtc_0, sc_embeddings_Gtc_0 = graph_spectral_clustering(Atc_0, k_clusters, shift_operator='laplacian', norm_type='sym', eig_type='U')

We infer that the silhouette diagrams that effectively fulfill the criteria for good clustering and offer pertinent insights into brain functioning are:
- $K=8$ ($SC=0.370$) for $\mathcal{G}$;
- $K=7$ ($SC=0.351$) for $\mathcal{H}_{II}$;
- $K=8$ ($SC=0.392$) for $\mathcal{H}_{TC}$.
  
The silhouette diagrams can be seen in the following cell.

In [ ]:
plot_models_silhouette_diagrams(list_kclusters=[8], list_embeddings=sc_embeddings_G, list_kmeans_models=sc_models_G,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}$',
                                               fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/silhouette_diagrams_sc_G.pdf')

plot_models_silhouette_diagrams(list_kclusters=[7], list_embeddings=sc_embeddings_Gii_0, list_kmeans_models=sc_models_Gii_0,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{II}^{(0)}$',
                                               fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/silhouette_diagrams_sc_Gii_0.pdf')

plot_models_silhouette_diagrams(list_kclusters=[8], list_embeddings=sc_embeddings_Gtc_0, list_kmeans_models=sc_models_Gtc_0,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{TC}^{(0)}$',
                                               fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/silhouette_diagrams_sc_Gtc_0.pdf')

In [ ]:
# Plotting pie charts of the selected spectral clustering models and the nodes distribution in each cluster according to the Schaefer's atlas subnets
 
plot_pie_clusters(models_list=sc_models_G, k_cluster=8, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'$\text{Nodes distribution of the spectral clustering model of the brain graph }\mathcal{G}$',
                  file_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/pie_charts_sc_G.pdf')

plot_pie_clusters(models_list=sc_models_Gii_0, k_cluster=7, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'$\text{Nodes distribution of the spectral clustering model of the brain hypergraph mode }\mathcal{G}_{II}^{(0)}$',
                  file_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/pie_charts_sc_Gii_0.pdf')

plot_pie_clusters(models_list=sc_models_Gtc_0, k_cluster=8, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'$\text{Nodes distribution of the spectral clustering model of the brain hypergraph mode }\mathcal{G}_{TC}^{(0)}$',
                  file_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/pie_charts_sc_Gtc_0.pdf')

In [ ]:
# Plotting a 3D visualization of the Schaefer's atlas for didatic purposes,
# where each ROI's centroid is accurately positioned within the three-dimensional Euclidean model space.

Plot_Brain_Subnets(title="Schaefer's atlas", movie='./Figures/brain_schaefer_atlas.html')

In [ ]:
# Plotting the 3D visualizations of the selected spectral clustering models on the brain.

Plot_Brain_Clusters(models_list=sc_models_G, k_cluster=8, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain graph $\mathcal{G}$',
                    movie='./Figures/From_Pairwise_to_Higher-order_Clustering/brain_clusters_sc_G.html')

Plot_Brain_Clusters(models_list=sc_models_Gii_0, k_cluster=7, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(0)}$',
                    movie='./Figures/From_Pairwise_to_Higher-order_Clustering/brain_clusters_sc_Gii_0.html')

Plot_Brain_Clusters(models_list=sc_models_Gtc_0, k_cluster=8, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(0)}$',
                    movie='./Figures/From_Pairwise_to_Higher-order_Clustering/brain_clusters_sc_Gtc_0.html')

Running the DANMF clustering algorithm on $\mathbf{A}$, $\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(0)}$ and $\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(0)}$ for $K\in\{2,3,\cdots,30\}$.

In [ ]:
k_clusters = range(7,9)
layers = [58]
pre_iterations=10
iterations=5000
lamb=0.1

g = normalize_matrix(A)
danmf_models_G, silhouette_score_models_danmf_G, danmf_embeddings_G = danmf_clustering(g, k_clusters, layers=layers, pre_iterations=pre_iterations, iterations=iterations, lamb=lamb)

g = normalize_matrix(Aii_0)
danmf_models_Gii_0, silhouette_score_models_danmf_Gii_0, danmf_embeddings_Gii_0 = danmf_clustering(g, k_clusters, layers=layers, pre_iterations=pre_iterations, iterations=iterations, lamb=lamb)

g = normalize_matrix(Atc_0)
danmf_models_Gtc_0, silhouette_score_models_danmf_Gtc_0, danmf_embeddings_Gtc_0 = danmf_clustering(g, k_clusters, layers=layers, pre_iterations=pre_iterations, iterations=iterations, lamb=lamb)

The selected DANMF clustering models are:
- $K=8$ ($SC=0.728$) for $\mathcal{G}$;
- $K=8$ ($SC=0.723$) for $\mathcal{H}_{II}$;
- $K=7$ ($SC=0.727$) for $\mathcal{H}_{TC}$.
  
The silhouette diagrams can be seen in the following cell.

In [ ]:
plot_models_silhouette_diagrams(list_kclusters=[8], list_embeddings=danmf_embeddings_G, list_kmeans_models=danmf_models_G,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}$',
                                               fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/silhouette_diagrams_danmf_G.pdf')

plot_models_silhouette_diagrams(list_kclusters=[8], list_embeddings=danmf_embeddings_Gii_0, list_kmeans_models=danmf_models_Gii_0,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{II}^{(0)}$',
                                               fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/silhouette_diagrams_danmf_Gii_0.pdf')

plot_models_silhouette_diagrams(list_kclusters=[7], list_embeddings=danmf_embeddings_Gtc_0, list_kmeans_models=danmf_models_Gtc_0,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{TC}^{(0)}$',
                                               fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/silhouette_diagrams_danmf_Gtc_0.pdf')

In [ ]:
# Plotting pie charts of the selected DANMF clustering models and the nodes distribution in each cluster according to the Schaefer's atlas subnets

plot_pie_clusters(models_list=danmf_models_G, k_cluster=8, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'$\text{Nodes distribution of the DANMF clustering model of the brain graph }\mathcal{G}$',
                  file_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/pie_charts_danmf_G.pdf')

plot_pie_clusters(models_list=danmf_models_Gii_0, k_cluster=8, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'$\text{Nodes distribution of the DANMF clustering model of the brain hypergraph mode }\mathcal{G}_{II}^{(0)}$',
                  file_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/pie_charts_danmf_Gii_0.pdf')

plot_pie_clusters(models_list=danmf_models_Gtc_0, k_cluster=7, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'$\text{Nodes distribution of the DANMF clustering model of the brain hypergraph mode }\mathcal{G}_{TC}^{(0)}$',
                  file_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/pie_charts_danmf_Gtc_0.pdf')

In [ ]:
# Plotting the 3D visualizations of the selected DANMF clustering models on the brain.

Plot_Brain_Clusters(models_list=danmf_models_G, k_cluster=8, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain graph $\mathcal{G}$',
                    movie='./Figures/From_Pairwise_to_Higher-order_Clustering/brain_clusters_danmf_G.html')

Plot_Brain_Clusters(models_list=danmf_models_Gii_0, k_cluster=8, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(0)}$',
                    movie='./Figures/From_Pairwise_to_Higher-order_Clustering/brain_clusters_danmf_Gii_0.html')

Plot_Brain_Clusters(models_list=danmf_models_Gtc_0, k_cluster=7, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(0)}$',
                    movie='./Figures/From_Pairwise_to_Higher-order_Clustering/brain_clusters_danmf_Gtc_0.html')

Analyzing the average of the tubal scalars of $\widehat{\mathcal{M}}_s^{abs}, \widehat{\mathcal{T}}_s^{abs}$:

In [ ]:
plot_mean_tubal_scalars(As_ii.fft,range(0,51), step_xticks=2, title=r'Mean squared root of the tubal scalar power spectral density of $\mathcal{M}_s$',
                        fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/mean_tubal_scalars_Ms.pdf')

plot_mean_tubal_scalars(As_tc.fft,range(0,51), step_xticks=2, title=r'Mean squared root of the tubal scalar power spectral density of $\mathcal{T}_s$',
                        fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/mean_tubal_scalars_Ts.pdf')

Defining $\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(4)}$ and $\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(4)}$, which are the adjacency matrices of $\mathcal{G}_{II}^{(4)}$ and $\mathcal{G}_{TC}^{(4)}$, respectively.

In [23]:
Aii_4 = np.abs(As_ii.fft[:,:,4])
Atc_4 = np.abs(As_tc.fft[:,:,4])

Running the spectral clustering algorithm on $\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(4)}$ and $\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(4)}$ for $K\in\{2,3,\cdots,30\}$.

In [24]:
k_clusters = range(2,31)

sc_models_Gii_4, silhouette_score_sc_models_Gii_4, sc_embeddings_Gii_4 = graph_spectral_clustering(Aii_4, k_clusters, shift_operator='laplacian', norm_type='sym', eig_type='U')
sc_models_Gtc_4, silhouette_score_sc_models_Gtc_4, sc_embeddings_Gtc_4 = graph_spectral_clustering(Atc_4, k_clusters, shift_operator='laplacian', norm_type='sym', eig_type='U')

The selected spectral clustering models are:
- $K=5$ ($SC=0.432$) for $\mathcal{H}_{II}$;
- $K=6$ ($SC=0.502$) for $\mathcal{H}_{TC}$.
  
The silhouette diagrams can be seen in the following cell.

In [ ]:
plot_models_silhouette_diagrams(list_kclusters=[5], list_embeddings=sc_embeddings_Gii_4, list_kmeans_models=sc_models_Gii_4,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{II}^{(4)}$',
                                               fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/silhouette_diagrams_sc_Gii_4.pdf')

plot_models_silhouette_diagrams(list_kclusters=[6], list_embeddings=sc_embeddings_Gtc_4, list_kmeans_models=sc_models_Gtc_4,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{TC}^{(4)}$',
                                               fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/silhouette_diagrams_sc_Gtc_4.pdf')

In [ ]:
# Plotting pie charts of the selected spectral clustering models and the nodes distribution in each cluster according to the Schaefer's atlas subnets

plot_pie_clusters(models_list=sc_models_Gii_4, k_cluster=5, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'$\text{Nodes distribution of the spectral clustering model of the brain hypergraph mode }\mathcal{G}_{II}^{(4)}$',
                  file_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/pie_charts_sc_Gii_4.pdf')

plot_pie_clusters(models_list=sc_models_Gtc_4, k_cluster=6, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'$\text{Nodes distribution of the spectral clustering model of the brain hypergraph mode }\mathcal{G}_{TC}^{(4)}$',
                  file_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/pie_charts_sc_Gtc_4.pdf')

In [ ]:
# Plotting the 3D visualizations of the selected spectral clustering models on the brain.

Plot_Brain_Clusters(models_list=sc_models_Gii_4, k_cluster=5, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(4)}$',
                    movie='./Figures/From_Pairwise_to_Higher-order_Clustering/brain_clusters_sc_Gii_4.html')

Plot_Brain_Clusters(models_list=sc_models_Gtc_4, k_cluster=6, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(4)}$',
                    movie='./Figures/From_Pairwise_to_Higher-order_Clustering/brain_clusters_sc_Gtc_4.html')

Running the DANMF clustering algorithm on $\left(\widehat{\mathbf{M}}_s^{abs}\right)^{(4)}$ and $\left(\widehat{\mathbf{T}}_s^{abs}\right)^{(4)}$ for $K\in\{2,3,\cdots,30\}$.

In [ ]:
k_clusters = range(4,6)
layers = [58]
pre_iterations=10
iterations=5000
lamb=0.1

g = normalize_matrix(Aii_4)
danmf_models_Gii_4, silhouette_score_models_danmf_Gii_4, danmf_embeddings_Gii_4 = danmf_clustering(g, layers=layers, k_clusters=k_clusters, pre_iterations=pre_iterations, iterations=iterations, lamb=lamb)

g = normalize_matrix(Atc_4)
danmf_models_Gtc_4, silhouette_score_models_danmf_Gtc_4, danmf_embeddings_Gtc_4 = danmf_clustering(g, layers=layers, k_clusters=k_clusters, pre_iterations=pre_iterations, iterations=iterations, lamb=lamb)

The selected DANMF clustering models are:
- $K=4$ ($SC=0.668$) for $\mathcal{H}_{II}$;
- $K=5$ ($SC=0.639$) for $\mathcal{H}_{TC}$.
  
The silhouette diagrams can be seen in the following cell.

In [ ]:
plot_models_silhouette_diagrams(list_kclusters=[4], list_embeddings=danmf_embeddings_Gii_4, list_kmeans_models=danmf_models_Gii_4,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{II}^{(4)}$',
                                               fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/silhouette_diagrams_danmf_Gii_4.pdf')

plot_models_silhouette_diagrams(list_kclusters=[5], list_embeddings=danmf_embeddings_Gtc_4, list_kmeans_models=danmf_models_Gtc_4,
                                               nodes_color=schaefer_nodes_color, name_dataset='$\mathcal{G}_{TC}^{(4)}$',
                                               fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/silhouette_diagrams_danmf_Gtc_4.pdf')

In [ ]:
# Plotting pie charts of the selected DANMF clustering models and the nodes distribution in each cluster according to the Schaefer's atlas subnets

plot_pie_clusters(models_list=danmf_models_Gii_4, k_cluster=4, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'$\text{Nodes distribution of the DANMF clustering model of the brain hypergraph mode }\mathcal{G}_{II}^{(4)}$',
                  file_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/pie_charts_danmf_Gii_4.pdf')

plot_pie_clusters(models_list=danmf_models_Gtc_4, k_cluster=5, nodes_color=schaefer_nodes_color, nodes_subnet=schaefer_nodes_subnet,
                  fig_title=r'$\text{Nodes distribution of the DANMF clustering model of the brain hypergraph mode }\mathcal{G}_{TC}^{(4)}$',
                  file_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/pie_charts_danmf_Gtc_4.pdf')

In [ ]:
# Plotting the 3D visualizations of the selected DANMF clustering models on the brain.

Plot_Brain_Clusters(models_list=danmf_models_Gii_4, k_cluster=4, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain hypergraph mode $\mathcal{G}_{II}^{(4)}$',
                    movie='./Figures/From_Pairwise_to_Higher-order_Clustering/brain_clusters_danmf_Gii_4.html')

Plot_Brain_Clusters(models_list=danmf_models_Gtc_4, k_cluster=5, nodes_color=schaefer_nodes_color, title=r'Clusters of the brain hypergraph mode $\mathcal{G}_{TC}^{(4)}$',
                    movie='./Figures/From_Pairwise_to_Higher-order_Clustering/brain_clusters_danmf_Gtc_4.html')

In [25]:
# Defining a Pandas Dataframe that contains all the selected clustering models and Schaefer's subnet labels.

df_cluster_labels_selected = pd.DataFrame({'subnet': schaefer_nodes_subnet,
                                           'schaefer': schaefer_nodes_label,
                                           'sc_G':get_clusters_label(sc_models_G, kcluster=8),
                                           'sc_Gii_0':get_clusters_label(sc_models_Gii_0, kcluster=7),
                                           'sc_Gtc_0':get_clusters_label(sc_models_Gtc_0, kcluster=8),
                                           'danmf_G': get_clusters_label(danmf_models_G, kcluster=8),
                                           'danmf_Gii_0': get_clusters_label(danmf_models_Gii_0, kcluster=8),
                                           'danmf_Gtc_0': get_clusters_label(danmf_models_Gtc_0, kcluster=7),
                                           'sc_Gii_4': get_clusters_label(sc_models_Gii_4, kcluster=5),
                                           'sc_Gtc_4': get_clusters_label(sc_models_Gtc_4, kcluster=6),
                                           'danmf_Gii_4': get_clusters_label(danmf_models_Gii_4, kcluster=4),
                                           'danmf_Gtc_4': get_clusters_label(danmf_models_Gtc_4, kcluster=5)})
print(df_cluster_labels_selected)

The following cell generate PDF files where the binarized adjacency matrices of $\mathcal{G}$, $\mathcal{G}^{(0)}_{II}$, $\mathcal{G}^{(0)}_{TC}$, $\mathcal{G}^{(4)}_{II}$ and $\mathcal{G}^{(4)}_{TC}$ and its modularities are plotted with distinct values of density $D\in\{5\%, 10\%, 15\%, 20\%\}$

In [ ]:
list_density = [0.05, 0.10, 0.15, 0.20]
df_modularity_sc = pd.DataFrame()
df_modularity_danmf = pd.DataFrame()
for density in list_density:

    mod_G_sc, mod_G_danmf = generate_binarized_adjacency_matrices(matrix=A, density=density, subnet_labels=df_cluster_labels_selected['subnet'],
                                        atlas_cluster_labels=df_cluster_labels_selected['schaefer'], sc_cluster_labels=df_cluster_labels_selected['sc_G'],
                                        danmf_cluster_labels=df_cluster_labels_selected['danmf_G'], matrix_name=r'$\mathcal{G}$',
                                        fig_name=f'./Binarized_Adjacency_Matrices/adjacency_matrix_G_binarized_{int(density*100)}%.pdf', plot_show=False)

    mod_Gii_0_sc, mod_Gii_0_danmf = generate_binarized_adjacency_matrices(matrix=Aii_0, density=density, subnet_labels=df_cluster_labels_selected['subnet'],
                                        atlas_cluster_labels=df_cluster_labels_selected['schaefer'], sc_cluster_labels=df_cluster_labels_selected['sc_Gii_0'],
                                        danmf_cluster_labels=df_cluster_labels_selected['danmf_Gii_0'], matrix_name=r'$\mathcal{G}_{II}^{(0)}$',
                                        fig_name=f'./Binarized_Adjacency_Matrices/adjacency_matrix_Gii_0_binarized_{int(density*100)}%.pdf', plot_show=False)

    mod_Gtc_0_sc, mod_Gtc_0_danmf = generate_binarized_adjacency_matrices(matrix=Atc_0, density=density, subnet_labels=df_cluster_labels_selected['subnet'],
                                        atlas_cluster_labels=df_cluster_labels_selected['schaefer'], sc_cluster_labels=df_cluster_labels_selected['sc_Gtc_0'],
                                        danmf_cluster_labels=df_cluster_labels_selected['danmf_Gtc_0'], matrix_name=r'$\mathcal{G}_{TC}^{(0)}$',
                                        fig_name=f'./Binarized_Adjacency_Matrices/adjacency_matrix_Gtc_0_binarized_{int(density*100)}%.pdf', plot_show=False)

    mod_Gii_4_sc, mod_Gii_4_danmf = generate_binarized_adjacency_matrices(matrix=Aii_4, density=density, subnet_labels=df_cluster_labels_selected['subnet'],
                                        atlas_cluster_labels=df_cluster_labels_selected['schaefer'], sc_cluster_labels=df_cluster_labels_selected['sc_Gii_4'],
                                        danmf_cluster_labels=df_cluster_labels_selected['danmf_Gii_4'], matrix_name=r'$\mathcal{G}_{II}^{(4)}$',
                                        fig_name=f'./Binarized_Adjacency_Matrices/adjacency_matrix_Gii_4_binarized_{int(density*100)}%.pdf', plot_show=False)

    mod_Gtc_4_sc, mod_Gtc_4_danmf = generate_binarized_adjacency_matrices(matrix=Atc_4, density=density, subnet_labels=df_cluster_labels_selected['subnet'],
                                        atlas_cluster_labels=df_cluster_labels_selected['schaefer'], sc_cluster_labels=df_cluster_labels_selected['sc_Gtc_4'],
                                        danmf_cluster_labels=df_cluster_labels_selected['danmf_Gtc_4'], matrix_name=r'$\mathcal{G}_{TC}^{(4)}$',
                                        fig_name=f'./Binarized_Adjacency_Matrices/adjacency_matrix_Gtc_4_binarized_{int(density*100)}%.pdf', plot_show=False)
    
    df_modularity_sc[f'D_{int(density * 100)}%'] = [mod_G_sc, mod_Gii_0_sc, mod_Gtc_0_sc, mod_Gii_4_sc, mod_Gtc_4_sc]
    df_modularity_danmf[f'D_{int(density * 100)}%'] = [mod_G_danmf, mod_Gii_0_danmf, mod_Gtc_0_danmf, mod_Gii_4_danmf, mod_Gtc_4_danmf]

df_modularity_sc.rename(index={0: 'G', 1: 'Gii_0', 2: 'Gtc_0', 3: 'Gii_4', 4: 'Gtc_4'}, inplace=True)
df_modularity_danmf.rename(index={0: 'G', 1: 'Gii_0', 2: 'Gtc_0', 3: 'Gii_4', 4: 'Gtc_4'}, inplace=True)
print(df_modularity_sc)
print(df_modularity_danmf)

Loads a Pandas Dataframe that consists of peak values at $k=0,4$ of the tubal scalars average of $\left(\widehat{\mathcal{M}_1}_s^{abs}\right)^{[n]}, \left(\widehat{\mathcal{T}_1}_s^{abs}\right)^{[n]}, \left(\widehat{\mathcal{M}_2}_s^{abs}\right)^{[n]}, \left(\widehat{\mathcal{T}_2}_s^{abs}\right)^{[n]}$ for each $n$-individual brain hypergraph

In [ ]:
df = pd.read_pickle('./df_peak_ii_tc_0_4_zscore_mean_tubal_scalars.pkl')
print(df.head(10))

Plotting the peak values correlation between the first and second resting-state scans of all individuals at $k=0.4$ for $II$ and $TC$.

In [ ]:
plot_correlation_scatters(df, figsize=(15,4), fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/correlation_peak_values.pdf')

Plotting the distribution of the peak values between males and females in each resting-state scan at $k=0,4$ for $II$ and $TC$.

In [ ]:
plot_metrics_distribution(df, figsize=(15,11), fig_dir='./Figures/From_Pairwise_to_Higher-order_Clustering/distribution_peak_values.pdf')